# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2023-spring@googlegroups.com

Slide:    [Link](https://docs.google.com/presentation/d/18LkR8qulwSbi3SVoLl1XNNGjQQ_qczs_35lrJWOmHCk/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/t/c76950cc460140eba30a576ca7668d28)

# Set up the environment


## Package installation

In [1]:
# Training progress bar
!pip install -q qqdm

## Downloading data

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh |  bash
!apt-get install -y --allow-unauthenticated git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.



The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.
Need to get 7419 kB of archives.
After this operation, 4936 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.3.0 [7419 kB]
Fetched 7419 kB in 1s (11.3 MB/s)
(Reading database ... 111522 files and directories currently installed.)
Preparing to unpack .../git-lfs_3.3.0_amd64.deb ...
Unpacking git-lfs (3.3.0) 

In [3]:
!git clone https://github.com/chiyuanhsiao/ml2023spring-hw8

Cloning into 'ml2023spring-hw8'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 2), reused 8 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), done.
Resolving deltas: 100% (2/2), done.


In [4]:
%cd ./ml2023spring-hw8
!git lfs install
!git lfs pull

/kaggle/working/ml2023spring-hw8
Updated Git hooks.
Git LFS initialized.


# Import packages

In [5]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd
from torchvision.transforms.functional import crop

# Loading data

In [6]:
train = np.load('./ml2023spring-hw8/trainingset.npy', allow_pickle=True)
test = np.load('./ml2023spring-hw8/testingset.npy', allow_pickle=True)

print(train.shape)
print(test.shape)

(100000, 64, 64, 3)
(19636, 64, 64, 3)


## Random seed
Set the random seed to a certain value for reproducibility.

In [7]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(48763)

# Autoencoder

# Models & loss

In [8]:
class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 64 * 3, 1024),
            nn.BatchNorm1d(1024),
            nn.PReLU(),
            nn.Linear(1024, 32),
            #nn.BatchNorm1d(512),
            #nn.PReLU(), 
            #nn.Linear(512, 256), 
            #nn.BatchNorm1d(256),
            #nn.PReLU(), 
            #nn.Linear(256, 32),
            # nn.ReLU(), 
            # nn.Linear(24, 12)
        )    # Hint: dimension of latent space can be adjusted
        
        self.decoder = nn.Sequential(
            # nn.Linear(12, 24),
            # nn.ReLU(), 
            nn.Linear(32, 1024),
            nn.BatchNorm1d(1024),
            nn.PReLU(), 
            nn.Linear(1024, 64*64*3),
            #nn.BatchNorm1d(512),
            #nn.PReLU(),
            #nn.Linear(512, 1024),
            #nn.BatchNorm1d(1024),
            #nn.PReLU(),
            #nn.Linear(1024, 64 * 64 * 3), 
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
# Output = ((I-K+2P)/S + 1), where
# I - a size of input neuron,
# K - kernel size,
# P - padding,
# S - stride.”
# “𝑊′=(𝑊−𝐹+2𝑃/𝑆)+1”
#  ((512-4+2*1)/2+1)=
class cnnfcn_autoencoder(nn.Module):
    def __init__(self):
        super(cnnfcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            #nn.Linear(64 * 64 * 3, 768),
            #nn.BatchNorm1d(768),
            #nn.PReLU(),
            nn.Conv2d(3, 128, 4, stride=2, padding=1), 
            nn.BatchNorm2d(128),
            nn.PReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1), 
            nn.BatchNorm2d(256),
            nn.PReLU(),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),   
            nn.BatchNorm2d(512),
            nn.PReLU(),
#             nn.MaxPool2d(kernel_size=4, stride=2),
            #nn.BatchNorm1d(512),
            #nn.PReLU(), 
            #nn.Linear(512, 256), 
            #nn.BatchNorm1d(256),
            #nn.PReLU(), 
            #nn.Linear(256, 32),
            # nn.ReLU(), 
            # nn.Linear(24, 12)
        )    # Hint: dimension of latent space can be adjusted
        self.encoder2 = nn.Sequential(
            nn.Linear(64 * 64 * 512, 1024),
            nn.BatchNorm1d(1024),
            nn.PReLU(),
            nn.Linear(1024, 32),
            
        )
        self.decoder2 = nn.Sequential(
            nn.Linear(32, 1024),
            nn.BatchNorm1d(1024),
            nn.PReLU(),
            nn.Linear(1024, 64 * 64 * 3),
            nn.Tanh(),
        )
        self.decoder1 = nn.Sequential(
            # nn.Linear(12, 24),
            # nn.ReLU(), 
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1),
            
            #nn.ReLU(),
            #nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
            #nn.MaxPool2d(kernel_size=4, stride=2)
            #nn.BatchNorm1d(512),
            #nn.PReLU(),
            #nn.Linear(512, 1024),
            #nn.BatchNorm1d(1024),
            #nn.PReLU(),
            #nn.Linear(1024, 64 * 64 * 3), 
        )

    def forward(self, x):
        x = self.encoder(x)
        #print(x.shape)
        #x = self.decoder1(x)
        print(x.shape)
        x = x.view(-1, 64*64*512)
        print(x.shape)
        x = self.encoder2(x)
        print(x.shape)
        x = self.decoder2(x)
#         x = x.view(-1,64,64,512)
        print(x.shape)
#         x = self.decoder1(x)
        return x
    
class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 128, 4, stride=2, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1), 
            nn.BatchNorm2d(256),
            nn.ReLU(),
            #nn.Conv2d(48, 96, 4, stride=2, padding=1),   # medium: remove this layer
            #nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1),
            
            #nn.ReLU(),
            #nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),    
            nn.ReLU(),
        )
        self.enc_out_1 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.enc_out_2 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
			      nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), 
            nn.ReLU(),
			      nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1), 
            nn.Tanh(),
        )

    def encode(self, x):
        h1 = self.encoder(x)
        return self.enc_out_1(h1), self.enc_out_2(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_vae(recon_x, x, mu, logvar, criterion):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    mse = criterion(recon_x, x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return mse + KLD

# Dataset module

Module for obtaining and processing data. The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].


In [9]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        # 與r11921a25討論
        self.transform = transforms.Compose([
            transforms.Lambda(lambda x: x.to(torch.float32)),
            transforms.Lambda(lambda x: crop(x,16,14,28,32)),
            transforms.Resize((64,64)),
            transforms.Lambda(lambda x: 2. * x/255. - 1.),
        ])
        
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

# Training

## Configuration


In [10]:
# Training hyperparameters
num_epochs = 80
batch_size = 512
learning_rate = 1e-3

# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size, num_workers=2, pin_memory=True)

# Model
model_type = 'fcn'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_classes = {'fcn': fcn_autoencoder(), 'cnn': conv_autoencoder(), 'vae': VAE(), 'cnnfcn' : cnnfcn_autoencoder()}
model = model_classes[model_type].cuda()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.9999)

## Training loop

In [11]:

best_loss = np.inf
model.train()

qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
for epoch in qqdm_train:
    tot_loss = list()
    for data in train_dataloader:

        # ===================loading=====================
        img = data.float().cuda()
        if model_type in ['fcn']:
            img = img.view(img.shape[0], -1)
        if model_type in ['cnnfcn']:
            img1 = img.view(img.shape[0], -1)
        # ===================forward=====================
        output = model(img)
        if model_type in ['vae']:
            loss = loss_vae(output[0], img, output[1], output[2], criterion)
        #elif model_type in ['cnnfcn']:
        #    loss = criterion(output, img.view(img.shape[0], -1))
        else:
            loss = criterion(output, img)

        tot_loss.append(loss.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
        print('epoch:' +f'{epoch + 1:.0f}/{num_epochs:.0f}' + ',loss: ' + f'{mean_loss:.4f}')
    # ===================log========================
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss': f'{mean_loss:.4f}',
    })
    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))

 Iters    Elapsed Time      Speed                                               
 0/80           -             -                                                 
Description   0.0% |                                                           |

epoch:1/80,loss: 0.0432


 Iters    Elapsed Time      Speed    epoch   loss                               
 1/80   00:00:28<00:37:17  0.04it/s  1/80   0.0432                              
Description   1.2% |                                                           |

epoch:2/80,loss: 0.0221


 Iters    Elapsed Time      Speed    epoch   loss                               
 2/80   00:00:55<00:35:56  0.04it/s  2/80   0.0221                              
Description   2.5% |█                                                          |

...

 Iters    Elapsed Time      Speed    epoch   loss                               
 76/80  00:31:27<00:01:39  0.04it/s  76/80  0.0136                              
Description  95.0% |████████████████████████████████████████████████████████   | Iters    Elapsed Time      Speed    epoch   loss                               
 77/80  00:31:50<00:01:14  0.04it/s  77/80  0.0137                              
Description  96.2% |████████████████████████████████████████████████████████   | Iters    Elapsed Time      Speed    epoch   loss                               
 78/80  00:32:13<00:00:49  0.04it/s  78/80  0.0136                              
Description  97.5% |█████████████████████████████████████████████████████████  |

epoch:79/80,loss: 0.0136


 Iters    Elapsed Time      Speed    epoch   loss                               
 79/80  00:32:37<00:00:24  0.04it/s  79/80  0.0136                              
Description  98.8% |██████████████████████████████████████████████████████████ |

epoch:80/80,loss: 0.0136


 Iters    Elapsed Time      Speed    epoch   loss                               
 80/80  00:33:01<00:00:00  0.04it/s  80/80  0.0136                              
Description 100.0% |███████████████████████████████████████████████████████████| Iters    Elapsed Time      Speed    epoch   loss                               
 80/80  00:33:01<00:00:00  0.04it/s  80/80  0.0136                              
Description 100.0% |███████████████████████████████████████████████████████████|

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

In [12]:
eval_batch_size = 200

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=2)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = f'last_model_{model_type}.pt'
model = torch.load(checkpoint_path)
model.eval()

# prediction file 
out_file = './prediction_v72.csv'

In [13]:
anomality = list()
with torch.no_grad():
  for i, data in enumerate(test_dataloader):
    img = data.float().cuda()
    if model_type in ['fcn']:
      img = img.view(img.shape[0], -1)
    if model_type in ['cnnfcn']:
      img1 = img.view(img.shape[0], -1)
    output = model(img)
    if model_type in ['vae']:
      output = output[0]
    if model_type in ['fcn']:
        loss = eval_loss(output, img).sum(-1)
    #elif model_type in ['cnnfcn']:
    #    loss = eval_loss(output, img.view(img.shape[0], -1)).sum(-1)
    else:
        loss = eval_loss(output, img).sum([1, 2, 3])
    anomality.append(loss)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['score'])
df.to_csv(out_file, index_label = 'ID')